### Import

In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

### GPU

In [2]:
torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [3]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono'
excl_folders = ['NoFX_mono', 'NoFX_mono_preprocessed']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)

# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [6]:
print(len(dataset))
print(dataset.mel_shape)
print(dataset.num_fx)
print(dataset.max_num_settings)
dataset.fx_to_label

163488
torch.Size([1, 128, 87])
14
6


{&#39;808&#39;: 0,
 &#39;BD2&#39;: 1,
 &#39;BMF&#39;: 2,
 &#39;DPL&#39;: 3,
 &#39;DS1&#39;: 4,
 &#39;FFC&#39;: 5,
 &#39;MGS&#39;: 6,
 &#39;MT2&#39;: 7,
 &#39;OD1&#39;: 8,
 &#39;RAT&#39;: 9,
 &#39;RBM&#39;: 10,
 &#39;SD1&#39;: 11,
 &#39;TS9&#39;: 12,
 &#39;VTB&#39;: 13}

### TRAIN SetCondNET

In [7]:
# model
setnet = models.SettingsNetCond(n_settings= dataset.max_num_settings,
                                mel_shape=dataset.mel_shape, 
                                num_embeddings=dataset.num_fx, 
                                embedding_dim=50)
# optimizer
optimizer = optim.Adam(setnet.parameters(), lr=0.001)
# loss function
loss_func = nn.MSELoss(reduction='mean')

In [8]:
print(setnet)

SettingsNetCond(
  (emb): Embedding(14, 50)
  (fc0): Linear(in_features=50, out_features=11136, bias=True)
  (conv1): Conv2d(2, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=6, bias=True)
)


In [9]:
# TRAIN and TEST SettingsNetCond OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_settings_cond_net(
        model=setnet,
        optimizer=optimizer, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_settings_cond_net(
        model=setnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function=loss_func, 
        device='cpu'
    )
    
    test_loss, test_correct, test_results = trainer.test_settings_cond_net(
        model=setnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function=loss_func, 
        device='cpu'
    )

    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[2000/117711 (2%)]	Total Loss: 8.6619	Avg Loss: 0.0043
Train Epoch: 0	[4000/117711 (3%)]	Total Loss: 11.3463	Avg Loss: 0.0028
Train Epoch: 0	[6000/117711 (5%)]	Total Loss: 12.9441	Avg Loss: 0.0022
Train Epoch: 0	[8000/117711 (7%)]	Total Loss: 14.3382	Avg Loss: 0.0018
Train Epoch: 0	[10000/117711 (8%)]	Total Loss: 15.6296	Avg Loss: 0.0016
Train Epoch: 0	[12000/117711 (10%)]	Total Loss: 16.8447	Avg Loss: 0.0014
Train Epoch: 0	[14000/117711 (12%)]	Total Loss: 18.0213	Avg Loss: 0.0013
Train Epoch: 0	[16000/117711 (14%)]	Total Loss: 19.1484	Avg Loss: 0.0012
Train Epoch: 0	[18000/117711 (15%)]	Total Loss: 20.2052	Avg Loss: 0.0011
Train Epoch: 0	[20000/117711 (17%)]	Total Loss: 21.2004	Avg Loss: 0.0011
Train Epoch: 0	[22000/117711 (19%)]	Total Loss: 22.1548	Avg Loss: 0.0010
Train Epoch: 0	[24000/117711 (20%)]	Total Loss: 23.0916	Avg Loss: 0.0010
Train Epoch: 0	[26000/117711 (22%)]	Total Loss: 23.9776	Avg Loss: 0.0009
Train Epoch: 0	[28000/117711 (24%)]	Total Loss: 24.8257	Avg L

In [10]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  83.76447400837645
Epoch:  49

Accuracy:  75.90029818793485
Epoch:  49

Accuracy:  76.13615511652088
Epoch:  49



In [11]:
# SAVE
models_folder = 'saved_models'
model_name = '20201007_setnetcond'
results_folder = 'saved_results'
results_subfolder = model_name

In [13]:
# SAVE MODEL
torch.save(setnet, '%s/%s' % (models_folder, model_name))

In [15]:
# SAVE RESULTS - all losses, all correct, best results
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
best_train_results_npy = np.array(all_train_results[49])

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
best_val_results_npy = np.array(all_val_results[49])

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
best_test_results_npy = np.array(all_test_results[49])

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_train_results')), arr=best_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_val_results')), arr=best_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_test_results')), arr=best_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)